In [1]:
import os
# import pandas library as pd
import pandas as pd
from bs4 import BeautifulSoup
from lxml import etree
from os import listdir
from os.path import isfile, join

# create an Empty DataFrame object
df = pd.DataFrame()
print(df)

#this function is being used in extract data funtion 
#to get columns per row and assigning them 
#if missing it will assign empty string 
def getTextFromTag(columns, index):
    columnText = ""
    try:
        columnText = columns[index].get_text()
    except:
        pass
    return columnText

#this is extract data function
#this function is callled in reports per day function to extract data 
#it reads one file and adds to data frame
#it takes data frame and returns data frame updated per files 
def extractdata(df, filename):

    with open(filename, 'r') as f:

        contents = f.read()
        #this is lxml method to read
        #soup = BeautifulSoup(contents, 'lxml')

        #this is beautiful soup html parser 
        soup = BeautifulSoup(contents, "html.parser")
        dom = etree.HTML(str(soup))

        print("File: " + soup.title.text)
        
        try:
            linerSize = dom.xpath('//*[@id="AutoNumber1"]/thead/tr[2]/td/table/tbody/tr[2]/td[2]/table/tbody/tr[2]/td[1]')[0].text
            date = dom.xpath('//*[@id="AutoNumber1"]/thead/tr[1]/td/table/tbody/tr[1]/td[2]/table/tbody/tr[3]/td')[0].text
            wellno = dom.xpath('//*[@id="AutoNumber1"]/thead/tr[1]/td/table/tbody/tr[1]/td[3]/table[1]/tbody/tr[2]/td')[0].text
            rigname = dom.xpath('//*[@id="AutoNumber1"]/thead/tr[1]/td/table/tbody/tr[1]/td[4]/table/tbody/tr[2]/td')[0].text
        except:
            linerSize = ""
            date = ""
            wellno = ""
            rigname = ""
        
        if linerSize is None:
            linerSize = "None"
        #assign soup items in to table and rows rows is table body and table rows 
        table = soup.find("table", {"id": "AutoNumber2"})     
        #printing the table
        #print(table)
                
        rows = table.find("tbody").find_all("tr")

        for row in rows:
            
            #this will grab all the columns for each row 
            columns = row.find_all("td")
            print(len(columns))

            
            lateral = getTextFromTag(columns, 4)
            holesize = getTextFromTag(columns, 5)
            majoroperation = getTextFromTag(columns, 7)
            action = getTextFromTag(columns, 8)
            operation = getTextFromTag(columns, 9)
            company = getTextFromTag(columns, 10)
            holeDepthStart = getTextFromTag(columns, 11)
            holeDepthEnd = getTextFromTag(columns, 12)
            eventDepthStart = getTextFromTag(columns, 13)
            eventDepthEnd = getTextFromTag(columns, 14)
            if len(columns) == 16:
                SummaryOfOperation = getTextFromTag(columns, 15)
            else: 
                SummaryOfOperation = getTextFromTag(columns, 17)
            #checking for lost time
                if company == "LT Summary":
                    continue
            data = {
                "Date": date,
                "Well No": wellno,
                "Liner Size": linerSize,
                "Operation": operation,
                "Rig Name": rigname,
                "lateral":lateral,
                "holesize":holesize,
                "majoroperation": majoroperation,
                "action": action,
                "Company Name": company,              
                "Hole Depth Start" : holeDepthStart,
                "Hole Depth End" : holeDepthEnd,
                "Event Depth Start" : eventDepthStart,
                "Event Depth End" : eventDepthEnd,
                "Summarry Of Operation" : SummaryOfOperation       
            }
            
            df = df.append(data, ignore_index = True)

    return df


def readReportsPerDay(df, MRday):

    mypath = MRday + "/reports"
    #mypath += "/reports"
    onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

    #print(type(onlyfiles))
    #print(onlyfiles)
    #print(os.getcwd())

    count = 1

    for filename in onlyfiles:

        
        print(len(filename))
        print("sf|", filename , "|", sep="")
        print("DS_Store"in filename)
        print("DS_Store" in "DS_Store")

        print(type(filename))

        if ".DS_Store" == filename:
            print("hello")
            continue

        if ".ipynb" in MRday or ".csv" in MRday or ".DS_Store" in MRday or ".xlsx" in MRday or ".DS_store" in MRday:
            continue


        filename = mypath + "/" + filename

        print(str(count) + " " + filename)

      #  df = extractdata(df, filename)
        count += 1

    return df


#this funtion will call the 
#
def runInRunningFolder(df):



    for MRday in listdir(os.getcwd()):

        if ".ipynb" in MRday or ".csv" in MRday or ".DS_Store" in MRday or ".xlsx" in MRday or ".DS_store" in MRday:
            continue

        print(MRday + "\n")

        df = readReportsPerDay(df, MRday)

    return df


print("Extract function is done, please continue.")



#to display to make sure we have empty data frame     
#display(df)      
print(df)



print("Setting up libraries, and empty data frame is done, please continue.\n")
print("-Yemane")


Empty DataFrame
Columns: []
Index: []
Extract function is done, please continue.
Empty DataFrame
Columns: []
Index: []
Setting up libraries, and empty data frame is done, please continue.

-Yemane


In [20]:
print(os.getcwd())

df = runInRunningFolder(df)

print(df)

/Users/Yemane/Documents/GitHub/YemaneClasses/pythonProject_Mac/NPT Extractor 
MR 4-15-2022

18
sf|ADC-35 MAQL-5.html|
False
True
<class 'str'>
1 MR 4-15-2022/reports/ADC-35 MAQL-5.html
9
sf|.DS_Store|
True
True
<class 'str'>
hello
21
sf|ADC-49 HRDH-2128.html|
False
True
<class 'str'>
2 MR 4-15-2022/reports/ADC-49 HRDH-2128.html
19
sf|088TE QTIF-176.html|
False
True
<class 'str'>
3 MR 4-15-2022/reports/088TE QTIF-176.html
19
sf|910TE DMMM-521.html|
False
True
<class 'str'>
4 MR 4-15-2022/reports/910TE DMMM-521.html
Empty DataFrame
Columns: []
Index: []


In [8]:
print(".DS_store" in ".DS_store")

True


In [10]:
#filename = my_Master_name 
filename = df.iloc[0][0]
filename = filename.replace("/", "-")
print(filename)
#print(my_master_name)

df.to_csv(filename + ".csv", sep=',', encoding='utf-8')
#df.to_csv(my_master_name + ".csv", sep=',', encoding='utf-8')

IndexError: single positional indexer is out-of-bounds

In [27]:
display(df)

""


In [ ]:
#this is for saving the file name 

In [12]:
df.head()

,Date,Well No,Liner Size,Operation,Rig Name,lateral,holesize,majoroperation,action,Company Name,column11,column12,column13,column14,column15,column16,Summarry Of Operation
0,02/27/2022,ZULF-106,4 1/2,MPMP,ADM-266,0,,CT,CIRC,RIG,0,0,,,CONT. SWEEP HOLE W/30 BBLS HI-VIS CHC & WORK B...,,
1,02/27/2022,ZULF-106,4 1/2,STRG,ADM-266,0,,CT,BR,RIG,0,0,6327,4494,"ATTEMPT TO POH W CO ASSY #4, OBSERVE 20 KLBS D...",,
2,02/27/2022,ZULF-106,4 1/2,MPMP,ADM-266,0,,CT,CIRC,RIG,0,0,,,"SWEEP & CIRC AT TOL, W 73 PCF HI-VIS MILL BRIN...",,
3,02/27/2022,ZULF-106,4 1/2,STDP,ADM-266,0,,CT,TO,RIG,0,0,4494,1945,"POH W 6-1/8"" CO ASSY #4, ON 4"" DP.* MONITOR WE...",,
4,02/27/2022,ZULF-106,4 1/2,BHA,ADM-266,0,,CT,HT,RIG,0,0,,,"POH, CLEAN & LD, XO + 3X 9-5/8"" SAMAA MAGNETS ...",,


In [13]:
df.tail()


,Date,Well No,Liner Size,Operation,Rig Name,lateral,holesize,majoroperation,action,Company Name,column11,column12,column13,column14,column15,column16,Summarry Of Operation
2004,02/27/2022,BRRI-780,None,WLT,HI-4,0,8 1/2,LOG,LOG,SWL,10575,10575,10224,10225,SLB TAKING SAMPLE @ (POINT #1) 10225' WIRELINE...,,
2005,02/27/2022,MRJN-733,None,STDP,SDA,0-1-1,8 1/2,DRLG,TO,RIG,11535,11535,7525,485,"POOH W/ 8-1/2"" BHI RSS/LWD BHA.",,
2006,02/27/2022,MRJN-733,None,NA,SDA,0-1-1,8 1/2,DRLG,PJSM,RIG,11535,11535,,,"FLOW CHECK, WELL STATIC.",,
2007,02/27/2022,MRJN-733,None,BHA,SDA,0-1-1,8 1/2,DRLG,HT,RIG,11535,11535,,,"HPJSM, COMPL. POOH & L/D 8-1/2"" BHI RSS/LWD D...",,
2008,02/27/2022,MRJN-733,None,NA,SDA,0-1-1,8 1/2,OT,SUSP,RIG,11535,11535,,,"SUSPENDED OPERATIONS ON 8-1/2"" SECTION.",,


In [ ]:
#use this for saving in range

In [ ]:
# myfilename = input ('please input the file name you want to save csf file: ')
# print(myfilename +" is in: "+os.getcwd())
# df.to_csv(myfilename + ".csv", sep=',', encoding='utf-8')

In [14]:
#filname = input("please put the name of the csv file you like to save: ")
#filename = my_Master_name 
filename = df.iloc[0][0]
filename = filename.replace("/", "-")
print(filename)
#print(my_master_name)

df.to_csv(filename + ".csv", sep=',', encoding='utf-8')
#df.to_csv(my_master_name + ".csv", sep=',', encoding='utf-8')

02-27-2022
